# Opt. Techniques with Text Data

In [1]:
# NLP Multi-Class Classification with DNN using Opt. Alg.

# Datasets
#  - AG News Classification Dataset : https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset & http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html

# Opt. Algorithms
    # - Stokastik Gradient Descent
    # - Stokastik Gradient Descent with Momentum
    # - Stokastik Adam

# AG News Dataset
    # - Class Index: Consists of 1-4 numbers which represents class of which 1-World, 2-Sports, 3-Business, 4-Sci/Tech
    # - Title: Contains Title of the Articles
    # - Description: Contains Description of the Articles
    # - Data Shape: Train: 120K & Test: 7600

### Import Libraries

In [1]:
import os
import warnings

from src.utils import get_project_config
from src.utils import missing_values
import pandas as pd
from dotenv import load_dotenv

In [2]:
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

warnings.filterwarnings('ignore')

#### Set Path and Constant Values

In [4]:
# Set Constants
ROOT_PATH = os.getcwd()
CFG_PATH = os.path.join(ROOT_PATH, 'cfg')
DATA_PATH = os.path.join(ROOT_PATH, 'data')

In [5]:
# import tensorflow as tf
# tf.config.list_physical_devices('GPU')

In [6]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

### Import Dataset

In [7]:
df_train_data = pd.read_csv(os.path.join(DATA_PATH, "train.csv.zip"), compression='zip')
df_train_data.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again."
1,3,Carlyle Looks Toward Commercial Aerospace (Reuters),"Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market."
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
3,3,Iraq Halts Oil Exports from Main Southern Pipeline (Reuters),"Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday."
4,3,"Oil prices soar to all-time record, posing new menace to US economy (AFP)","AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections."


In [8]:
df_test_data = pd.read_csv(os.path.join(DATA_PATH, "test.csv.zip"), compression='zip')
df_test_data.head()

,Class Index,Title,Description
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.
1,4,The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com),"SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket."
2,4,Ky. Company Wins Grant to Study Peptides (AP),"AP - A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins."
3,4,Prediction Unit Helps Forecast Wildfires (AP),"AP - It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar."
4,4,Calif. Aims to Limit Farm-Related Smog (AP),"AP - Southern California's smog-fighting agency went after emissions of the bovine variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure."


In [9]:
df_train_data_orj = df_train_data.copy()
df_test_data_orj = df_test_data.copy()

### Columns Transformation

In [10]:
df_data_list = [df_train_data, df_test_data]
class_map = {1:'World', 2:'Sports', 3:'Business', 4:'Sci/Tech'}

for df in df_data_list:
    # print("DataFrame Name:", df)
    df.columns = df.columns.str.replace(' ', '', regex=False)
    df['ClassName'] = df['ClassIndex'].map(class_map)

In [11]:
df_train_data['Text'] = df_train_data['Title'] + ' ' + df_train_data['Description']
df_test_data['Text'] = df_test_data['Title'] + ' ' + df_test_data['Description']

In [12]:
df_train_data.head()

,ClassIndex,Title,Description,ClassName,Text
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.",Business,"Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again."
1,3,Carlyle Looks Toward Commercial Aerospace (Reuters),"Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.",Business,"Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market."
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.,Business,Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
3,3,Iraq Halts Oil Exports from Main Southern Pipeline (Reuters),"Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.",Business,"Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday."
4,3,"Oil prices soar to all-time record, posing new menace to US economy (AFP)","AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections.",Business,"Oil prices soar to all-time record, posing new menace to US economy (AFP) AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections."


### Show Metadata, Shape, Statistics and Missing Table

In [13]:
df_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ClassIndex   120000 non-null  int64 
 1   Title        120000 non-null  object
 2   Description  120000 non-null  object
 3   ClassName    120000 non-null  object
 4   Text         120000 non-null  object
dtypes: int64(1), object(4)
memory usage: 4.6+ MB


In [14]:
df_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7600 entries, 0 to 7599
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ClassIndex   7600 non-null   int64 
 1   Title        7600 non-null   object
 2   Description  7600 non-null   object
 3   ClassName    7600 non-null   object
 4   Text         7600 non-null   object
dtypes: int64(1), object(4)
memory usage: 297.0+ KB


In [15]:
df_train_data.shape, df_test_data.shape

((120000, 5), (7600, 5))

In [16]:
df_train_data['ClassIndex'].value_counts()

ClassIndex
3    30000
4    30000
2    30000
1    30000
Name: count, dtype: int64

In [17]:
df_test_data['ClassIndex'].value_counts()

ClassIndex
3    1900
4    1900
2    1900
1    1900
Name: count, dtype: int64

In [18]:
missing_values(df=df_train_data, threshold=0, asc_sorting=False)

,Count,Percent
Columns,,
ClassIndex,0,0.0
Title,0,0.0
Description,0,0.0
ClassName,0,0.0
Text,0,0.0


In [19]:
missing_values(df=df_test_data, threshold=0, asc_sorting=False)

,Count,Percent
Columns,,
ClassIndex,0,0.0
Title,0,0.0
Description,0,0.0
ClassName,0,0.0
Text,0,0.0


### Text Data Preprocessing

#### Text Preprocess

In [21]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

# NLTK kaynaklarını indir
nltk.download('stopwords')
nltk.download('wordnet')

# HTML etiketlerini ve URL'leri temizleme fonksiyonu
def remove_html_urls(text):
    """HTML etiketlerini ve URL'leri kaldırır."""
    text = re.sub(r'<.*?>', '', text)  # HTML etiketlerini temizleme
    text = re.sub(r'http\S+|www\S+', '', text)  # URL'leri temizleme
    return text

# Ekstra boşlukları temizleme
def remove_extra_whitespace(text):
    """Metindeki gereksiz boşlukları kaldırır."""
    text = re.sub(r'\s+', ' ', text)  # Birden fazla boşluğu tek bir boşluğa dönüştürür
    return text.strip()  # Başlangıç ve sondaki boşlukları siler

# Expand correction (yaygın kısaltmaların düzeltilmesi)
# def expand_contractions(text):
#     """Metindeki yaygın kısaltmaları düzeltir."""
#     contractions = {
#         "can't": "cannot",
#         "won't": "will not",
#         "don't": "do not",
#         "I'm": "I am",
#         "he's": "he is",
#         "she's": "she is",
#         "it's": "it is",
#         "we're": "we are",
#         "they're": "they are",
#         # Eklemek istenilen diğer kısaltmalar
#     }
#     for contraction, expanded in contractions.items():
#         text = text.replace(contraction, expanded)
#     return text
# Expand correction (yaygın kısaltmaların düzeltilmesi)
def expand_contractions(text):
    """Yaygın İngilizce kısaltmaları açar."""
    contractions = {
        "i'm": "i am", "don't": "do not", "can't": "cannot", "won't": "will not",
        "it's": "it is", "he's": "he is", "she's": "she is", "they're": "they are",
        "we're": "we are", "you've": "you have", "i've": "i have", "that's": "that is",
        "there's": "there is", "couldn't": "could not", "shouldn't": "should not"
        # Burada gerekli kısaltmaları ekleyebilirsin
    }
    for contraction, expansion in contractions.items():
        text = text.replace(contraction, expansion)
    return text

# Preprocessing fonksiyonu
def preprocess_text(df:pd.DataFrame, column_name:str, lemmatize:bool=True, use_stemming:bool=False):
    # 1. Küçük harfe dönüştürme
    df[column_name] = df[column_name].str.lower()
    
    # 2. HTML etiketlerini ve URL'leri temizleme
    df[column_name] = df[column_name].apply(remove_html_urls)
    
    # 3. Ekstra boşlukları kaldırma
    df[column_name] = df[column_name].apply(remove_extra_whitespace)
    
    # 4. Kısaltmaları düzeltme
    df[column_name] = df[column_name].apply(expand_contractions)
    
    # 5. Özel karakterlerin ve sayıların temizlenmesi
    df[column_name] = df[column_name].str.replace(r'[^a-zA-Z\s]', '', regex=True)
    
    # 6. Stop words kaldırma
    stop_words = set(stopwords.words('english'))
    df[column_name] = df[column_name].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    
    # 7. Lemmatization ya da Stemming
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        df[column_name] = df[column_name].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
    elif use_stemming:
        stemmer = PorterStemmer()
        df[column_name] = df[column_name].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

    return df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\metin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\metin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df_train_data = preprocess_text(df=df_train_data, column_name="Text")
df_test_data = preprocess_text(df=df_test_data, column_name="Text")

In [ ]:
df_train_data.head(3)

In [ ]:
df_test_data.head(3)

#### Tokenization & Padding

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 10000
# max_length = 200 

# Prepare tokenizer for text processing
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_train_data['Description'])

# Convert texts to sequences
X_train = tokenizer.texts_to_sequences(df_train_data['Description'])
X_test = tokenizer.texts_to_sequences(df_test_data['Description'])

# Pad sequences to ensure uniform length
max_sequence_length = 100  # Adjust based on your dataset's text length

X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_test = pad_sequences(X_test, maxlen=max_sequence_length)

In [29]:
X_train[0].shape, X_train[1].shape, X_train[2].shape

((100,), (100,), (100,))

In [30]:
X_test[0].shape, X_test[1].shape, X_test[2].shape

((100,), (100,), (100,))

In [31]:
# Sınıf etiketlerini one-hot encoding yapıyoruz
y_train = to_categorical(df_train_data['ClassIndex'] - 1, num_classes=4)
y_test = to_categorical(df_test_data['ClassIndex'] - 1, num_classes=4)

### Model Architecture

CIFAR-10 veri kümesindeki 10 sınıfı sınıflandırmak için CNN mimarisi kullanılmıştır. CIFAR-10 veri kümesi, 32x32 çözünürlüğünde ve 3 kanallı (RGB) renklere sahip 10 sınıf içerir.

CNN Mimarisi
- 1.Katman: 32 filtre kullanarak 3x3 boyutunda bir kernel uygular. 'relu' aktivasyon fonksiyonu kullanılır.
- 2.Katman: İlk katmanın çıkışını girdi olarak alır daha derin özellikler çıkarır.
- MaxPooling: 2x2 havuzlama, boyut azaltma.
- Dropout: Eğitim sırasında %25 oranında nöronları rastgele devre dışı bırakır (overfitting önleme).
- 3.Katman: 64 filtre kullanarak daha fazla özellik çıkarır.
- 4.Katman: Ardışık Conv2D katmanları ile özellik çıkarımı güçlendirilir.
- MaxPooling: 2x2 havuzlama, boyut azaltma.
- Dropout: Yine %25 oranında nöronları devre dışı bırakarak aşırı öğrenme ihtimalini azaltır.
- Flatten: Çıkış verisini tek bir vektör haline getirir. Bu vektör, tam bağlantılı (Dense) katman için giriş olur.
- Dense: 512 nöronlu fully connected katman.
- Dropout: %50 oranında nöronları rastgele devre dışı bırakır. Modelin genelleme yeteneğini artırır.
- Çıkış Katmanı: 10 sınıflı, 'softmax' ile her bir sınıfa ait olasılık tahmini.

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten

def create_model(input_dim=10000, output_dim=4, input_length=max_sequence_length):
    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=128, input_length=input_length),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(output_dim, activation='softmax')
    ])
    # model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [43]:
# 5 farklı başlangıç noktası oluşturuyoruz

initial_weights = []
for _ in range(5):
    model = create_model()
    model.build(input_shape=(None, max_sequence_length))
    initial_weights.append(model.get_weights())

In [48]:
# initial_weights[0]
# len(initial_weights)

In [49]:
model.build(input_shape=(None, max_sequence_length))
model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_16 (Embedding)             │ (None, 100, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_16 (Flatten)                 │ (None, 12800)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_32 (Dense)                     │ (None, 128)                 │       1,638,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,919,044 (11.14 MB)

 Trainable params: 2,919,044 (11.14 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
# Optimizasyon algoritmaları ve parametreleri tanımlıyoruz
optimizer_configs = {
    'SGD': {'class': SGD, 'params': {'learning_rate': 0.01}},
    'SGD_momentum': {'class': SGD, 'params': {'learning_rate': 0.01, 'momentum': 0.9}},
    'Adam': {'class': Adam, 'params': {'learning_rate': 0.001}}
}

### Optimizations Algorithms

In [53]:
# Eğitim parametrelerini tanımlıyoruz
batch_size = 32
epochs = 10

# Sonuçları saklamak için listeleri tutuyoruz
trajectories = {}
history_records = {}
time_records = {}

for opt_name, opt_config in optimizer_configs.items():
    print(f"\nOptimizasyon Algoritması: {opt_name}")
    trajectories[opt_name] = []
    history_records[opt_name] = []
    time_records[opt_name] = []
    for idx, w0 in enumerate(initial_weights):
        print(f"\nBaşlangıç Noktası: {idx+1}")
        # Modeli başlangıç ağırlıklarıyla yeniden oluştur
        model = create_model()
        model.build(input_shape=(None, max_sequence_length))
        model.set_weights(w0)

        # Her model için optimizer'ı yeniden oluştur #aynı optimizer'ı kullanırken hata verdiği için eklendi
        optimizer_class = opt_config['class']
        optimizer_params = opt_config['params']
        optimizer = optimizer_class(**optimizer_params)

        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        # Ağırlık yörüngelerini saklamak için liste tutuyoruz
        weight_trajectory = []
        epoch_times = []
        history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}

        # **Başlangıç ağırlıklarını kaydet**
        initial_weights_vector = np.concatenate([w.flatten() for w in model.get_weights()])
        weight_trajectory.append(initial_weights_vector)

        for epoch in range(epochs):
            print(f"\n=== {opt_name} | Başlangıç Noktası {idx+1} | Epoch {epoch+1}/{epochs} ===")
            start_time = time.time()
            # Eğitimi bir epoch için gerçekleştir (her bir epoch sonunda ağırlıkları kaydedip eklemek için)
            model.fit(X_train, y_train, batch_size=batch_size, epochs=1, validation_data=(X_test, y_test), verbose=1)
            end_time = time.time()
            # Epoch süresini kaydet
            epoch_times.append(end_time - start_time)
            # Ağırlıkları kaydet
            weights = np.concatenate([w.flatten() for w in model.get_weights()])
            weight_trajectory.append(weights)
            # Kayıp ve doğruluğu kaydet
            loss, acc = model.evaluate(X_train, y_train, verbose=0)
            val_loss, val_acc = model.evaluate(X_test, y_test, verbose=0)
            history['loss'].append(loss)
            history['accuracy'].append(acc)
            history['val_loss'].append(val_loss)
            history['val_accuracy'].append(val_acc)

        # Sonuçları sakla
        trajectories[opt_name].append(np.array(weight_trajectory))
        history_records[opt_name].append(history)
        time_records[opt_name].append(np.array(epoch_times))


Optimizasyon Algoritması: SGD

Başlangıç Noktası: 1

=== SGD | Başlangıç Noktası 1 | Epoch 1/10 ===
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - accuracy: 0.2802 - loss: 1.3813 - val_accuracy: 0.3974 - val_loss: 1.3522

=== SGD | Başlangıç Noktası 1 | Epoch 2/10 ===
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.3872 - loss: 1.3253 - val_accuracy: 0.5825 - val_loss: 1.1070

=== SGD | Başlangıç Noktası 1 | Epoch 3/10 ===
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 23s 6ms/step - accuracy: 0.5948 - loss: 1.0239 - val_accuracy: 0.7425 - val_loss: 0.7380

=== SGD | Başlangıç Noktası 1 | Epoch 4/10 ===
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 23s 6ms/step - accuracy: 0.7614 - loss: 0.6797 - val_accuracy: 0.8237 - val_loss: 0.5190

=== SGD | Başlangıç Noktası 1 | Epoch 5/10 ===
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - accuracy: 0.8290 - loss: 0.5007 - val_accuracy: 0.8516 - val_loss: 0.4371

=== SGD | Başlangıç Noktası 1 | Epoch 6/10 ===
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - accuracy: 0.8

**learning_rate**: Öğrenme oranı, optimizasyon sürecinin adım büyüklüğünü belirler.learning_rate='auto' olarak ayarlandığında, t-SNE veri setinin
boyutuna göre uygun bir öğrenme oranı seçer.  

In [57]:
# TEST ET
# 7. Yörüngelerin t-SNE ile 2 Boyuta İndirgenmesi

# Tüm yörüngeleri birleştir
all_trajectories = []
labels = []

for idx in range(len(initial_weights)):
    for opt_name in optimizer_configs.keys():
        traj = trajectories[opt_name][idx]
        all_trajectories.append(traj)
        labels.append(opt_name)

# Yörüngeleri birleştir ve şekillendir
all_points = np.concatenate(all_trajectories, axis=0)

# t-SNE uygulaması
# n_iter=1000
tsne = TSNE(n_components=2, random_state=42, perplexity=50, n_iter=250, learning_rate='auto', init='pca', verbose=1)
tsne_results = tsne.fit_transform(all_points)

# Her yörüngenin başlangıç ve bitiş indekslerini bul
trajectory_lengths = [len(traj) for traj in all_trajectories]

[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Indexed 165 samples in 0.307s...


MemoryError: bad allocation

In [ ]:
# TEST ET
# 8. Yörüngelerin Görselleştirilmesi

# Renkleri belirle
colors = {'SGD': 'r', 'SGD_momentum': 'g', 'Adam': 'b'}

plt.figure(figsize=(12, 8))

start_idx = 0
for i, (opt_name, traj_len) in enumerate(zip(labels, trajectory_lengths)):
    end_idx = start_idx + traj_len
    x = tsne_results[start_idx:end_idx, 0]
    y = tsne_results[start_idx:end_idx, 1]
    # Her yörüngeyi çiz
    plt.plot(x, y, color=colors[opt_name],marker='o', label=opt_name if i % 5 == 0 else "", alpha=0.7)
    start_idx = end_idx

# Legend'da sadece algoritmaları göstermek için benzersiz etiketleri alalım
handles, labels_unique = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels_unique, handles))
plt.legend(by_label.values(), by_label.keys())

plt.title('Optimizasyon Yörüngeleri')
plt.xlabel('t-SNE Bileşen 1')
plt.ylabel('t-SNE Bileşen 2')
plt.show()

In [ ]:
# TEST ET
# 8. Yörüngelerin Görselleştirilmesi

# Renkleri belirle
colors = {'SGD': 'r', 'SGD_momentum': 'g', 'Adam': 'b'}

plt.figure(figsize=(12, 8))

start_idx = 0
for i, (opt_name, traj_len) in enumerate(zip(labels, trajectory_lengths)):
    end_idx = start_idx + traj_len
    x = tsne_results[start_idx:end_idx, 0]
    y = tsne_results[start_idx:end_idx, 1]
    # Her yörüngeyi çiz
    plt.plot(x, y, color=colors[opt_name], label=opt_name if i % 5 == 0 else "", alpha=0.7)
    start_idx = end_idx

# Legend'da sadece algoritmaları göstermek için benzersiz etiketleri alalım
handles, labels_unique = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels_unique, handles))
plt.legend(by_label.values(), by_label.keys())

plt.title('Optimizasyon Yörüngeleri')
plt.xlabel('t-SNE Bileşen 1')
plt.ylabel('t-SNE Bileşen 2')
plt.show()

In [ ]:
# Epoch vs Loss
# Optimizasyon algoritmalarının isimlerini alıyoruz
optimizer_names = list(optimizer_configs.keys())

# 3 tane alt grafik
num_algorithms = len(optimizer_names)

# Alt grafiklerin boyutlarını ayarlıyoruz
fig, axes = plt.subplots(nrows=1, ncols=num_algorithms, figsize=(6*num_algorithms, 5), sharey=True)

# Eğer tek bir grafik varsa, axes'i liste haline getirelim
if num_algorithms == 1:
    axes = [axes]

for idx, opt_name in enumerate(optimizer_names):
    ax = axes[idx]
    # Her algoritma için tüm başlangıç noktalarındaki eğitim geçmişini çizeceğiz
    for history in history_records[opt_name]:
        # Eğitim kaybı (Train Loss)
        ax.plot(history['loss'], linestyle='-', color=colors[opt_name],
                label='Train' if history == history_records[opt_name][0] else "")
        # Doğrulama kaybı (Validation Loss)
        ax.plot(history['val_loss'], linestyle='--', color=colors[opt_name],
                label='Validation' if history == history_records[opt_name][0] else "")

    ax.set_title(f'{opt_name} - Epoch vs Loss')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Epoch vs Accuracy
# Alt grafiklerin boyutlarını ayarlıyoruz
fig, axes = plt.subplots(nrows=1, ncols=num_algorithms, figsize=(6*num_algorithms, 5), sharey=True)

# Eğer tek bir grafik varsa, axes'i liste haline getirelim
if num_algorithms == 1:
    axes = [axes]

for idx, opt_name in enumerate(optimizer_names):
    ax = axes[idx]
    # Her algoritma için tüm başlangıç noktalarındaki eğitim geçmişini çizeceğiz
    for history in history_records[opt_name]:
        # Eğitim doğruluğu (Train Accuracy)
        ax.plot(history['accuracy'], linestyle='-', color=colors[opt_name],
                label='Train' if history == history_records[opt_name][0] else "")
        # Doğrulama doğruluğu (Validation Accuracy)
        ax.plot(history['val_accuracy'], linestyle='--', color=colors[opt_name],
                label='Validation' if history == history_records[opt_name][0] else "")

    ax.set_title(f'{opt_name} - Epoch vs Accuracy')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy')
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Time vs Loss
# Alt grafiklerin boyutlarını ayarlıyoruz
fig, axes = plt.subplots(nrows=1, ncols=num_algorithms, figsize=(6*num_algorithms, 5), sharey=True)

# Eğer tek bir grafik varsa, axes'i liste haline getirelim
if num_algorithms == 1:
    axes = [axes]

for idx, opt_name in enumerate(optimizer_names):
    ax = axes[idx]
    # Her algoritma için tüm başlangıç noktalarındaki eğitim geçmişini çizeceğiz
    for idx_hist, (history, times) in enumerate(zip(history_records[opt_name], time_records[opt_name])):
        cumulative_times = np.cumsum(times)
        # Eğitim kaybı (Train Loss)
        ax.plot(cumulative_times, history['loss'], linestyle='-', color=colors[opt_name],
                label='Train' if idx_hist == 0 else "")
        # Doğrulama kaybı (Validation Loss)
        ax.plot(cumulative_times, history['val_loss'], linestyle='--', color=colors[opt_name],
                label='Validation' if idx_hist == 0 else "")

    ax.set_title(f'{opt_name} - Time vs Loss')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Loss')
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
#### Epoch vs Loss Grafiği (Tümü)

plt.figure(figsize=(12, 8))

for opt_name in optimizer_configs.keys():
    for history in history_records[opt_name]:
        # Eğitim kaybı (Train Loss)
        plt.plot(history['loss'], linestyle='-', color=colors[opt_name],
                 label=f"{opt_name} - Train" if history == history_records[opt_name][0] else "")
        # Doğrulama kaybı (Validation Loss)
        plt.plot(history['val_loss'], linestyle='--', color=colors[opt_name],
                 label=f"{opt_name} - Validation" if history == history_records[opt_name][0] else "")

plt.title('Epoch vs Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
#### Time vs Loss Grafiği (TÜMÜ)

plt.figure(figsize=(12, 8))

for opt_name in optimizer_configs.keys():
    for idx, (history, times) in enumerate(zip(history_records[opt_name], time_records[opt_name])):
        cumulative_times = np.cumsum(times)
        # Eğitim kaybı (Train Loss)
        plt.plot(cumulative_times, history['loss'], linestyle='-', color=colors[opt_name],
                 label=f"{opt_name} - Train" if idx == 0 else "")
        # Doğrulama kaybı (Validation Loss)
        plt.plot(cumulative_times, history['val_loss'], linestyle='--', color=colors[opt_name],
                 label=f"{opt_name} - Validation" if idx == 0 else "")

plt.title('Time vs Loss')
plt.xlabel('Time (s)')
plt.ylabel('Loss')
plt.legend()
plt.show()
